In [1]:
import os
import tensorflow as tf
from sklearn import preprocessing
import text_cnn
import data_helper
from utils import parse_reuters, remove_minor_classes, topics_with_occurences_gt

/Users/hieuhuynh/Apps/anaconda3/envs/text-cls-py3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [6]:
_train_text_lst, _train_label_lst, _test_text_lst, _test_label_lst = parse_reuters()

# remove classes with very few occurrences
topics_selected = topics_with_occurences_gt(3, _train_label_lst)
train_text_lst, train_label_lst, test_text_lst, test_label_lst = remove_minor_classes(topics_selected, _train_text_lst, _train_label_lst, _test_text_lst, _test_label_lst)

# label transformer
label_binarizer = preprocessing.MultiLabelBinarizer(topics_selected)
label_binarizer.fit(train_label_lst)
y_train = label_binarizer.transform(train_label_lst)
y_test = label_binarizer.transform(test_label_lst)

processing file ../data/reut2-000.sgm
processing file ../data/reut2-001.sgm
processing file ../data/reut2-002.sgm
processing file ../data/reut2-003.sgm
processing file ../data/reut2-004.sgm
processing file ../data/reut2-005.sgm
processing file ../data/reut2-006.sgm
processing file ../data/reut2-007.sgm
processing file ../data/reut2-008.sgm
processing file ../data/reut2-009.sgm
processing file ../data/reut2-010.sgm
processing file ../data/reut2-011.sgm
processing file ../data/reut2-012.sgm
processing file ../data/reut2-013.sgm
processing file ../data/reut2-014.sgm
processing file ../data/reut2-015.sgm
processing file ../data/reut2-016.sgm
processing file ../data/reut2-017.sgm
processing file ../data/reut2-018.sgm
processing file ../data/reut2-019.sgm
processing file ../data/reut2-020.sgm
processing file ../data/reut2-021.sgm
Removed 58 topics with occurrences greater than 3. remain: 77 topics
Removing samples with minor topics
Train samples: 7773
Test samples: 3016


In [7]:
# step 1
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split
import numpy as np
max_document_length = max([len(x.split(' ')) for x in train_text_lst])
print('The maximum length of all sentences: {}'.format(max_document_length))
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x_train = np.array(list(vocab_processor.fit_transform(train_text_lst)))
x_test = np.array(list(vocab_processor.transform(test_text_lst)))
print('x train shape: {}'.format(x_train.shape))
print('x test shape: {}'.format(x_test.shape))

The maximum length of all sentences: 1267
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
x train shape: (7773, 1267)
x test shape: (3016, 1267)


In [8]:
# store data as pkl files
import pickle as pkl
with open('x_train.pkl', 'wb') as xtf:
    pkl.dump(x_train, xtf)
    xtf.close()
with open('x_test.pkl', 'wb') as xtesf:
    pkl.dump(x_test, xtesf)
    xtesf.close()
with open('y_train.pkl', 'wb') as ytf:
    pkl.dump(y_train, ytf)
    ytf.close()
with open('y_test.pkl', 'wb') as ytesf:
    pkl.dump(y_test, ytesf)
    ytesf.close()

with open('topics_selected.pkl', 'wb') as tselectedf:
    pkl.dump(topics_selected, tselectedf)
    tselectedf.close()
# Save the word_to_id map since predict.py needs it
vocab_processor.save(os.path.join(os.getcwd(), "vocab.pickle"))

In [2]:
# load data from pkl files
import pickle as pkl
with open('x_train.pkl', 'rb') as xtf:
    x_train = pkl.load(xtf)
    xtf.close()
with open('x_test.pkl', 'rb') as xtesf:
    x_test = pkl.load(xtesf)
    xtesf.close()
with open('y_train.pkl', 'rb') as ytf:
    y_train = pkl.load(ytf)
    ytf.close()
with open('y_test.pkl', 'rb') as ytesf:
    y_test = pkl.load(ytesf)
    ytesf.close()
    
with open('topics_selected.pkl', 'rb') as tselectedf:
    topics_selected = pkl.load(tselectedf)
    tselectedf.close()

with open('vocab.pickle', 'rb') as vocabf:
    vocab_processor = pkl.load(vocabf)
    vocabf.close()
print('x train shape: {}'.format(x_train.shape))
print('x test shape: {}'.format(x_test.shape))
print('y train shape: {}'.format(y_train.shape))
print('y test shape: {}'.format(y_test.shape))

x train shape: (7773, 1267)
x test shape: (3016, 1267)
y train shape: (7773, 77)
y test shape: (3016, 77)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
shuffle_indices = np.random.permutation(np.arange(len(y_train)))
x_shuffled = x_train[shuffle_indices]
y_shuffled = y_train[shuffle_indices]
x_train, x_dev, y_train, y_dev = train_test_split(x_shuffled, y_shuffled, test_size=0.1)

In [4]:
graph = tf.Graph()
session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
sess = tf.Session(config=session_conf)
cnn = text_cnn.TextCNN(
    sequence_length=x_train.shape[1],
    num_classes=y_train.shape[1],
    vocab_size=len(vocab_processor.vocabulary_),
    embedding_size=50,
    filter_sizes=[3, 4, 5],
    num_filters=32,
    l2_reg_lambda=0.0)
global_step = tf.Variable(0, name="global_step", trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate=1e-3,
                                           global_step=global_step, 
                                           decay_steps=300,
                                           decay_rate=0.95, staircase=True)
optimizer = tf.train.AdamOptimizer(1e-3)
grads_and_vars = optimizer.compute_gradients(cnn.loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
saver = tf.train.Saver()

In [5]:
def train_step(x_batch, y_batch):
    feed_dict = {
        cnn.input_x: x_batch,
        cnn.input_y: y_batch,
        cnn.dropout_keep_prob: 0.5}
    _, step, loss = sess.run([train_op, global_step, cnn.loss], feed_dict)
    return loss
# One evaluation step: evaluate the model with one batch
def dev_step(x_batch, y_batch):
    feed_dict = {cnn.input_x: x_batch, cnn.input_y: y_batch, cnn.dropout_keep_prob: 1.0}
    step, scores, loss = sess.run([global_step, cnn.scores, cnn.loss], feed_dict)
    predicted_labels_threshold, predicted_values_threshold, y_batch_pred = data_helper.get_label_using_scores_by_threshold(scores=scores, threshold=0.5)
    
    cur_rec_ts, cur_acc_ts, cur_F_ts = 0.0, 0.0, 0.0
    
    for index, predicted_label_threshold in enumerate(predicted_labels_threshold):
        rec_inc_ts, acc_inc_ts = data_helper.cal_metric(predicted_label_threshold, y_batch[index])
        cur_rec_ts, cur_acc_ts = cur_rec_ts + rec_inc_ts, cur_acc_ts + acc_inc_ts
        
    cur_rec_ts = cur_rec_ts / len(y_batch)
    cur_acc_ts = cur_acc_ts / len(y_batch)
    cur_F_ts = data_helper.cal_F(cur_rec_ts, cur_acc_ts)
    return cur_rec_ts, cur_acc_ts, cur_F_ts, loss, y_batch_pred

In [6]:
sess.run(tf.global_variables_initializer())

# Step 6: train the cnn model with x_train and y_train (batch by batch)
batch_size = 32
num_epochs = 30
train_batches = data_helper.batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
best_accuracy, best_at_step = 0, 0

In [7]:
evaluate_every = 200
checkpoint_every = 500
num_batches_per_epoch = int((len(x_train) - 1) / batch_size) + 1
for train_batch in train_batches:
    x_train_batch, y_train_batch = zip(*train_batch)
    train_loss = train_step(x_train_batch, y_train_batch)
    current_step = tf.train.global_step(sess, global_step)
    if current_step % 50 == 0:
        print('Train step %d loss: %.4f' % (current_step, train_loss))
    
    """Step 6.1: evaluate the model with x_dev and y_dev (batch by batch)"""
    if current_step % evaluate_every == 0:
        print('---------- Evaluate on step %d -----------' % current_step)
        dev_batches = data_helper.batch_iter(list(zip(x_dev, y_dev)), 32, 1)
        eval_counter, eval_loss, eval_rec_ts, eval_acc_ts, eval_F_ts = 0, 0.0, 0.0, 0.0, 0.0
        for dev_batch in dev_batches:
            x_dev_batch, y_dev_batch = zip(*dev_batch)
            cur_rec_ts, cur_acc_ts, cur_F_ts, cur_loss, _ = dev_step(x_dev_batch, y_dev_batch)
            # update ts scores
            eval_counter += 1
            eval_rec_ts, eval_acc_ts = eval_rec_ts + cur_rec_ts, eval_acc_ts + cur_acc_ts
            eval_loss += cur_loss
        # calculate metrics on the whole dev set
        eval_loss = float(eval_loss / eval_counter)
        eval_rec_ts = float(eval_rec_ts / eval_counter)
        eval_acc_ts = float(eval_acc_ts / eval_counter)
        eval_F_ts = data_helper.cal_F(eval_rec_ts, eval_acc_ts)

        print('loss on dev set: {}'.format(eval_loss))
        print('recall on dev set: {}'.format(eval_rec_ts))
        print('accuracy on dev set: {}'.format(eval_acc_ts))
        print('F score on dev set: {}'.format(eval_F_ts))

    """Step 6.2: save the model if it is the best based on accuracy on dev set"""
    if current_step % checkpoint_every == 0:
        checkpoint_prefix = os.path.join("checkpoint", "model")
        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
        print('Saved model to {}'.format(path))
    if current_step % num_batches_per_epoch == 0:
        current_epoch = current_step // num_batches_per_epoch
        print(" Epoch {} has finished!".format(current_epoch))

Train step 50 loss: 7.9856
Train step 100 loss: 6.9490
Train step 150 loss: 5.3841
Train step 200 loss: 4.5441
---------- Evaluate on step 200 -----------
loss on dev set: 4.658696460723877
recall on dev set: 0.45379166666666665
accuracy on dev set: 0.46
F score on dev set: 0.4568747435137477
 Epoch 1 has finished!
Train step 250 loss: 5.1319
Train step 300 loss: 5.0333
Train step 350 loss: 3.0060
Train step 400 loss: 5.0801
---------- Evaluate on step 400 -----------
loss on dev set: 4.310470924377442
recall on dev set: 0.48595833333333327
accuracy on dev set: 0.49424999999999997
F score on dev set: 0.49006909670563226
 Epoch 2 has finished!
Train step 450 loss: 5.3404
Train step 500 loss: 3.2028
Saved model to checkpoint/model-500
Train step 550 loss: 4.4943
Train step 600 loss: 4.7353
---------- Evaluate on step 600 -----------
loss on dev set: 4.035959920883179
recall on dev set: 0.5017916666666666
accuracy on dev set: 0.50925
F score on dev set: 0.5054933237172883
Train step 650 l

In [8]:
"""Step 7: predict x_test (batch by batch)"""
from sklearn import metrics
test_batches = data_helper.batch_iter(list(zip(x_test, y_test)), batch_size=32, num_epochs=1)
y_preds, y_test_reform = [], []
for test_batch in test_batches:
    x_test_batch, y_test_batch = zip(*test_batch)
    _, _, _, _, y_pred_batch = dev_step(x_test_batch, y_test_batch)
    y_preds.append(y_pred_batch)
    y_test_reform.append(y_test_batch)
y_preds = np.vstack(y_preds)
y_test_reform = np.vstack(y_test_reform)
report = metrics.classification_report(y_test_reform, y_preds, target_names=topics_selected)
print(report)

                 precision    recall  f1-score   support

          cocoa       1.00      0.78      0.88        18
        plywood       0.00      0.00      0.00         0
            tea       0.00      0.00      0.00         4
         coffee       1.00      0.89      0.94        28
            gas       0.00      0.00      0.00        17
         lumber       0.00      0.00      0.00         6
            gnp       0.94      0.49      0.64        35
            wpi       1.00      0.30      0.46        10
           earn       0.95      0.95      0.95      1088
            oat       0.00      0.00      0.00         6
         nickel       0.00      0.00      0.00         1
       palm-oil       0.00      0.00      0.00        10
            hog       1.00      0.33      0.50         6
    instal-debt       0.00      0.00      0.00         1
           corn       0.88      0.39      0.54        56
         barley       0.00      0.00      0.00        14
           lead       0.00    

/Users/hieuhuynh/Apps/anaconda3/envs/text-cls-py3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/hieuhuynh/Apps/anaconda3/envs/text-cls-py3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
print(y_preds.shape)
print(x_test.shape, y_test.shape)

(3016, 77)
(3016, 1267) (3016, 77)
